In [96]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [97]:
#function to make sure the date is a valid date for sorting data and not one from the future
#this function will add the year onto the date as it currently doesn't have it 
def adjust_date(row):
    
    season_year = row['Season']
    month = row['Date'].split(' ')[0] 
    day = row['Date'].split(' ')[1]
    
    month_numbers = {
        "January": 1, "February": 2, "March": 3, "April": 4,
        "May": 5, "June": 6, "July": 7, "August": 8,
        "September": 9, "October": 10, "November": 11, "December": 12
    }
    month_number = month_numbers[month]

    #adjusting for january and february games as the season rolls into the next year
    year = season_year + 1 if month_number in [1, 2] else season_year

    # Return the adjusted date in MM/DD/YYYY format
    return f"{month_number:02d}/{day}/{year}"

In [98]:
#cleaning data to prepare it for machine learning
#matches.dtypes
matches = pd.read_csv('matches.csv')

#Removing playoff row error due to differing season lengths
#playoffIdx = match_df[ (match_df['Date'] == 'Playoffs')].index
#match_df =  match_df.drop(playoffIdx)
#match_df = match_df.reset_index(drop = 'true')
#match_df.to_csv('matches.csv', index=False, encoding='utf-8')
#match_df


#Removing playoff row error due to differing season lengths
playoffIdx = matches[ (matches['Date'] == 'Playoffs')].index
matches =  matches.drop(playoffIdx)
matches = matches.reset_index(drop = 'true')
matches.to_csv('matches.csv', index=False, encoding='utf-8')

#converting dates to proper dates so that they can be stored as date types
matches['Date'] = matches.apply(adjust_date, axis=1)
matches['Date'] = pd.to_datetime(matches['Date'])


#converting home/away into numeric data
matches['Home/AwayCode'] = matches['Home/Away'].astype('category').cat.codes #0 is away, 1 is home



#converting team into numeric data
#matches['TeamCode'] = matches['Team'].astype('category').cat.codes

#converting opponents into numeric data
#matches['OppCode'] = matches['Opp'].astype('category').cat.codes

#adding hour column 
matches['Hour'] = matches['Time'].str.replace(':.+', '', regex = True).astype('int')


#adding day of the week column
matches['DayCode'] = matches['Date'].dt.dayofweek
#NOTE STILL NEED TO CLEAN UP DATE FIRST


#converting all other stats from objects to ints
for column in matches.columns[10:24]:
    matches[column] = pd.to_numeric(matches[column])

#adding in team year and opp year column to make teams separate by year
matches['TeamYear'] = matches['Team'].astype('str') + '' + matches['Season'].apply(str)
matches['OppYear'] = matches['Opp'].astype('str') + '' + matches['Season'].apply(str)


matches['TeamCode'] = matches['TeamYear'].astype('category').cat.codes

matches['OppCode'] = matches['OppYear'].astype('category').cat.codes

#converting Win/Loss to numbers
matches['Target'] = (matches['Result'] == 'W').astype('int')

In [103]:
#Initializing a random forest machine learning model
rf = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, random_state = 1)

#Training on previous games from this and the past two seasons
training = matches[matches['Date'] < '12-24-2024']
train = matches[matches['Date'] > '04-04-2019'] #delete this and change above line to fix
test = matches[matches['Date'] > '12-24-2024']

#Testing using 3 basic predictors, whether the game is home or away,
#who the team playing is and who the opponent is
#team's and oppoenents are broken down by year so the 2023 49ers are different than the 2024 49ers
#this helps to account for the NFL's volatility in team strength season to season
predictors = ['Home/AwayCode', 'TeamCode', 'OppCode']
rf.fit(train[predictors], train['Target'])
preds = rf.predict(test[predictors])

In [104]:
acc = accuracy_score(test['Target'], preds)
acc

0.890625

In [105]:
precision_score(test['Target'],preds)

0.6956521739130435

In [106]:
def mapWin(col):
    col.apply(lambda x : 'W' if x == 1 else 'L')

In [107]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('Date')
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [108]:
def make_predictions(data, predVals):
    rf = RandomForestClassifier(n_estimators = 1000, min_samples_split = 20, random_state = 1)
    training = data[data['Date'] < '04-04-2024']
    train = training[training['Date'] > '04-04-2019'] #cuts off the data from only the past two seasons as it tends to make the model more accurate surprisingly
    testing = data[data['Date'] > '04-04-2024']
    test = testing[testing['Date']<'12-24-2024']
    rf.fit(train[predVals], train['Target'])
    preds = rf.predict(test[predVals])
    combined = pd.DataFrame(dict(actual=test['Target'], prediction=preds))
    precision = precision_score(test['Target'],preds)
    accuracy = accuracy_score(test['Target'], preds)
    return combined, precision, accuracy, rf

In [109]:

#Used to bypass deprecation warning but still have the 'Team' index included in the dataframe
cols = ['TmSc', 'OppSc', 'Off1stD', 'OffTotYd', 'OffPassY',
       'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY',
       'DefTO', 'EPOff', 'EPDef', 'EPSp.']
new_cols = [f'{col}Rolling' for col in cols]
matches_rolling = matches.set_index('Team').groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols),include_groups = False)
matches_rolling = matches_rolling.droplevel(0)
matches_rolling = matches_rolling.reset_index()



predictorTests = []
predictorTests.append([])
predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 
                       'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.'])
predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO'])
predictorTests.append(['OffTotYd', 'DefTotYd', 'OffRushY', 'OffTO'])

predictorTests.append(['OffTotYd', 'DefTotYd', 'OffRushY', 'OffTO', 'DefTO'])


predictorTests.append(['TmSc', 'OppSc']) #should always be 100% as TmSc > OppSc = win
predictorTests.append(['EPOff', 'EPDef', 'EPSp.'])
predictorTests.append(['Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])
predictorTests.append(['OffTotYdRolling', 'DefTotYdRolling', 'OffRushYRolling', 'OffTORolling'])

predictorTests.append(['Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])

predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 
                       'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])

predictorTests.append(['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 
                       'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling',
                       'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling'])



#predictorTests.append(['Home/AwayCode', 'OppCode', 'TeamCode'])




rf_models = []
predictors = ['Home/AwayCode', 'TeamCode', 'OppCode']
for predictorCols in predictorTests:
    new_predictors = predictors + predictorCols
    combined, precision, accuracy, rf = make_predictions(matches_rolling, new_predictors)
    rf_models.append(rf)
    print("Predictors:", new_predictors)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")

    #This shows how many times a correct prediction was made when one team was predicted to win and the other was predicted to lose
    combined = combined.merge(matches_rolling[['Date','Team','Opp','Result']], left_index = True, right_index = True)
    merged = combined.merge(combined, left_on = ['Date', 'Team'], right_on = ['Date', 'Opp'])
    #Prints out the importance of the different features used to predict games
    feature_importances = rf.feature_importances_
    print("Feature Importances")
    for feature, importance in zip(new_predictors, feature_importances):
        print(f"{feature}: {importance}") 

    #remapping 0's and 1's to losses and wins
    mapping_cols = ['actual_x', 'actual_y', 'prediction_x', 'prediction_y']
    for col in mapping_cols:
        merged[col] = merged[col].apply(lambda x : 'W' if x == 1 else 'L')
        

    #prints correct win predictions
    print(merged[(merged['prediction_x'] == 'W') & (merged['prediction_y'] == 'L')]['actual_x'].value_counts())
    
    print("-"*200)
    print("")

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode']
Accuracy: 61.67%
Precision: 61.76%
Feature Importances
Home/AwayCode: 0.020520966918451497
TeamCode: 0.48971104008746796
OppCode: 0.48976799299408064
actual_x
W    144
L     88
Name: count, dtype: int64
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.']
Accuracy: 88.33%
Precision: 88.33%
Feature Importances
Home/AwayCode: 0.0009303754794104767
TeamCode: 0.013737644738599158
OppCode: 0.013150326846525295
Off1stD: 0.028563887862601042
OffTotYd: 0.051629938800152514
OffPassY: 0.01868793405136653
OffRushY: 0.030056504291857373
OffTO: 0.07184604602216758
Def1stD: 0.030620791954302437
DefTotYd: 0.051406

In [110]:
def joinGames(game_list):
    # Create a unique game identifier by sorting team names
    game_list['game_id'] = game_list.apply(lambda row: '_'.join(sorted([row['Team'], row['Opp']])), axis=1)
    
    # Keep only the first occurrence of each game
    unique_games = game_list.drop_duplicates(subset=['game_id'])
    
    # Create a cleaner output format
    result = pd.DataFrame({
        'Date': unique_games['Date'],
        'Home_Team': unique_games['Team'],
        'Away_Team': unique_games['Opp'],
        'Predicted_Winner': unique_games.apply(
            lambda row: row['Team'] if row['TeamPrediction'] == 'W' else row['Opp'],
            axis=1
        )
    })
    
    # Sort by date
    result = result.sort_values('Date').reset_index(drop=True)
    
    return result


In [126]:
## get week 18 games
week18 = matches_rolling[matches_rolling['Season'] == 2024].copy(deep = True)
week18 = week18[week18['Week'] == 18]
rf_idx = 0
predictors = ['Home/AwayCode', 'TeamCode', 'OppCode']
#predictorCols = ['Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd',
#'DefPassY', 'DefRushY', 'DefTO', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling',
#'OffTORolling', 'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling']
    
for predictorCols in predictorTests:
    new_predictors = predictors + predictorCols
    rf = rf_models[rf_idx]
    rf_idx += 1
    week18_preds = rf.predict_proba(week18[new_predictors])#predict win percentages
    week18_table = pd.DataFrame({'Predicted': week18_preds[:,1]}, index=week18.index) 
    #week18_table['Predicted'] = mapWin(week18_table['Predicted'])
    week18_table = week18_table.merge(week18[['Date', 'Team', 'Opp']], left_index=True, right_index=True)
    #week18_table['Predicted'] = week18_table['Predicted'].apply(lambda x : 'W' if x==1 else 'L')
    
    #week18_table = week18_table[ week18_table['Date'] <= pd.Timestamp.now()]
    print("Predictors:", new_predictors)
    merged18 = week18_table.merge(week18_table, left_on= ['Date', 'Team'], right_on = ['Date','Opp'])

    merged18 = merged18.rename(columns = {'Team_x':'Team', 'Predicted_x':'OrigTeamWinPct', 'Opp_x':'Opp', 'Predicted_y':'OrigOppWinPct'})


    #Normalizing win percentages so they add up to 1
    merged18['TeamWinPct'] = merged18['OrigTeamWinPct'] / (merged18['OrigTeamWinPct'] + merged18['OrigOppWinPct'])
    merged18['OppWinPct'] = merged18['OrigOppWinPct'] / (merged18['OrigTeamWinPct'] + merged18['OrigOppWinPct'])

    merged18['TeamPrediction'] = merged18['TeamWinPct'].apply(lambda x : 'W' if x > 0.5 else ('L' if x < 0.5 else 'D'))
    merged18['OppPrediction'] = merged18['OppWinPct'].apply(lambda x : 'W' if x > 0.5 else ('L' if x < 0.5 else 'D'))

    

    
    #merged18 = combined18.merge(combined, on= ['Date', 'Team'])

    #remapping 0's and 1's to losses and wins
    #mapping_cols = ['TeamPrediction', 'OppPrediction']
    #for col in mapping_cols:
    #    merged18[col] = merged18[col].apply(lambda x : 'W' if x == 1 else 'L')

    #merged18 = joinGames(merged18) #used to condense tables after analyzing the prediction set for them.




    #COMMENT THIS OUT TO SEE MORE DETAILS ON THE ORIGINAL WIN PERCENTAGE PREDICTIONS
    merged18 = merged18.reindex(columns = ['Date', 'Team', 'TeamPrediction', 'TeamWinPct', 'Opp', 'OppPrediction', 'OppWinPct'])

    #AND THEN UNCOMMENT TO SEE MORE DETAILS ON THE ORIGINAL WIN PERCENTAGE PREDICTIONS
    #merged18 = merged18.reindex(columns = ['Date', 'Team', 'TeamPredictionLetter', 'TeamPrediction', 'NormalizedTeamPrediction', 'Opp', 'OppPredictionLetter', 'OppPrediction', 'NormalizedOppPrediction'])
    

    
    merged18 = merged18.style.set_properties(**{'text-align': 'center'})

    display(merged18)

        
    #print(merged18[(merged18['Predicted'] == 'W') & (merged18['prediction_y'] == 'L')]['actual_x'].value_counts())
    print("-"*200)
    print("")

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode']


,Date,Team,TeamPrediction,TeamWinPct,Opp,OppPrediction,OppWinPct
0,2025-01-05 00:00:00,Arizona Cardinals,L,0.207556,San Francisco 49ers,W,0.792444
1,2025-01-05 00:00:00,Atlanta Falcons,W,0.818217,Carolina Panthers,L,0.181783
2,2025-01-04 00:00:00,Baltimore Ravens,W,0.693751,Cleveland Browns,L,0.306249
3,2025-01-05 00:00:00,Buffalo Bills,W,0.510402,New England Patriots,L,0.489598
4,2025-01-05 00:00:00,Carolina Panthers,L,0.181783,Atlanta Falcons,W,0.818217
5,2025-01-05 00:00:00,Chicago Bears,L,0.256104,Green Bay Packers,W,0.743896
6,2025-01-04 00:00:00,Cincinnati Bengals,L,0.394429,Pittsburgh Steelers,W,0.605571
7,2025-01-04 00:00:00,Cleveland Browns,L,0.306249,Baltimore Ravens,W,0.693751
8,2025-01-05 00:00:00,Dallas Cowboys,W,0.687384,Washington Commanders,L,0.312616
9,2025-01-05 00:00:00,Denver Broncos,L,0.363486,Kansas City Chiefs,W,0.636514


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.']


,Date,Team,TeamPrediction,TeamWinPct,Opp,OppPrediction,OppWinPct
0,2025-01-05 00:00:00,Arizona Cardinals,L,0.496300,San Francisco 49ers,W,0.503700
1,2025-01-05 00:00:00,Atlanta Falcons,L,0.499542,Carolina Panthers,W,0.500458
2,2025-01-04 00:00:00,Baltimore Ravens,W,0.500635,Cleveland Browns,L,0.499365
3,2025-01-05 00:00:00,Buffalo Bills,L,0.497053,New England Patriots,W,0.502947
4,2025-01-05 00:00:00,Carolina Panthers,W,0.500458,Atlanta Falcons,L,0.499542
5,2025-01-05 00:00:00,Chicago Bears,L,0.498907,Green Bay Packers,W,0.501093
6,2025-01-04 00:00:00,Cincinnati Bengals,L,0.498136,Pittsburgh Steelers,W,0.501864
7,2025-01-04 00:00:00,Cleveland Browns,L,0.499365,Baltimore Ravens,W,0.500635
8,2025-01-05 00:00:00,Dallas Cowboys,W,0.500641,Washington Commanders,L,0.499359
9,2025-01-05 00:00:00,Denver Broncos,L,0.498413,Kansas City Chiefs,W,0.501587


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO']


...

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'EPOff', 'EPDef', 'EPSp.', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling', 'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling']


,Date,Team,TeamPrediction,TeamWinPct,Opp,OppPrediction,OppWinPct
0,2025-01-05 00:00:00,Arizona Cardinals,L,0.497327,San Francisco 49ers,W,0.502673
1,2025-01-05 00:00:00,Atlanta Falcons,L,0.498989,Carolina Panthers,W,0.501011
2,2025-01-04 00:00:00,Baltimore Ravens,W,0.512998,Cleveland Browns,L,0.487002
3,2025-01-05 00:00:00,Buffalo Bills,W,0.500566,New England Patriots,L,0.499434
4,2025-01-05 00:00:00,Carolina Panthers,W,0.501011,Atlanta Falcons,L,0.498989
5,2025-01-05 00:00:00,Chicago Bears,L,0.492507,Green Bay Packers,W,0.507493
6,2025-01-04 00:00:00,Cincinnati Bengals,L,0.498347,Pittsburgh Steelers,W,0.501653
7,2025-01-04 00:00:00,Cleveland Browns,L,0.487002,Baltimore Ravens,W,0.512998
8,2025-01-05 00:00:00,Dallas Cowboys,W,0.500726,Washington Commanders,L,0.499274
9,2025-01-05 00:00:00,Denver Broncos,L,0.497752,Kansas City Chiefs,W,0.502248


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Predictors: ['Home/AwayCode', 'TeamCode', 'OppCode', 'Off1stD', 'OffTotYd', 'OffPassY', 'OffRushY', 'OffTO', 'Def1stD', 'DefTotYd', 'DefPassY', 'DefRushY', 'DefTO', 'Off1stDRolling', 'OffTotYdRolling', 'OffPassYRolling', 'OffRushYRolling', 'OffTORolling', 'Def1stDRolling', 'DefTotYdRolling', 'DefPassYRolling', 'DefRushYRolling', 'DefTORolling']


,Date,Team,TeamPrediction,TeamWinPct,Opp,OppPrediction,OppWinPct
0,2025-01-05 00:00:00,Arizona Cardinals,L,0.498032,San Francisco 49ers,W,0.501968
1,2025-01-05 00:00:00,Atlanta Falcons,W,0.500679,Carolina Panthers,L,0.499321
2,2025-01-04 00:00:00,Baltimore Ravens,W,0.503455,Cleveland Browns,L,0.496545
3,2025-01-05 00:00:00,Buffalo Bills,L,0.490675,New England Patriots,W,0.509325
4,2025-01-05 00:00:00,Carolina Panthers,L,0.499321,Atlanta Falcons,W,0.500679
5,2025-01-05 00:00:00,Chicago Bears,L,0.491256,Green Bay Packers,W,0.508744
6,2025-01-04 00:00:00,Cincinnati Bengals,L,0.494204,Pittsburgh Steelers,W,0.505796
7,2025-01-04 00:00:00,Cleveland Browns,L,0.496545,Baltimore Ravens,W,0.503455
8,2025-01-05 00:00:00,Dallas Cowboys,L,0.499780,Washington Commanders,W,0.500220
9,2025-01-05 00:00:00,Denver Broncos,W,0.500204,Kansas City Chiefs,L,0.499796


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

